In [ ]:
from __future__ import print_function
import os
from skimage.io import imsave
import argparse
import os
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
from torch.autograd import Variable
from IPython.display import Image
from skimage.io import imread
from skimage.transform import resize
import numpy as np
import glob

from ipywidgets import FloatSlider, IntSlider, Button
from ipywidgets import interact, interactive, fixed, interact_manual
from collections import OrderedDict

from IPython.display import display,clear_output
from ipywidgets import widgets
from collections import OrderedDict
%matplotlib inline
import matplotlib.pyplot as plt

from machinedesign.viz import grid_of_images_default, horiz_merge

import PIL.Image
from io import BytesIO
import IPython.display
import numpy as np
def showarray(a, fmt='png'):
    a = np.uint8(a)
    f = BytesIO()
    PIL.Image.fromarray(a).save(f, fmt)
    IPython.display.display(IPython.display.Image(data=f.getvalue()))

In [ ]:
!ls

In [ ]:
dataset = 'shoes'

G = torch.load('{}/gen.th'.format(dataset), map_location=lambda storage, loc: storage)
nz = G.main[0].weight.size(0)

In [ ]:
z_train = np.load('{}/bin.npz'.format(dataset))['X']

In [ ]:
def update(w):
    batch_size = 1
    clear_output()
    z = [w[str(i)].value if str(i) in w else 0 for i in range(nz)]
    z = np.array(z)
    z = z[np.newaxis, :, np.newaxis, np.newaxis]
    z = z.astype(np.float32)
    z = torch.from_numpy(z)
    z = z.repeat(batch_size, 1, 1, 1)
    z = Variable(z)
    fake = G(z)
    im = fake.data
    im = im.cpu().numpy()
    im = im[0]
    im = im.transpose((1, 2, 0))
    if im.shape[2] == 1:
        im = im[:, :, 0]
    if im.min()<0:
        im = 0.5*(im + 1)
    plt.imshow(im, cmap='gray')

w = OrderedDict()
for i in range(0, nz):
    w[str(i)] = IntSlider(min=0, max=1, value=0)

def click_train(bt):
    ind = np.random.randint(0, len(z_train))
    z = z_train[ind]
    for i in range(nz):
        w[str(i)].value = z[i]
    update(w)
    
def click_rand(bt):
    z = (np.random.uniform(size=nz) <= 0.5).astype('int32')
    for i in range(nz):
        w[str(i)].value = z[i]
    update(w)

w['train'] = Button(description='Random train')
w['train'].on_click(click_train)
w['random'] = Button(description='Random')
w['random'].on_click(click_rand)

w['display'] = Button(description='Display')
w['display'].on_click(lambda _:update(w))
widget_list = [w for name, w in w.items() if name in ('train', 'display', 'random') or int(name)<nz]
vbox = widgets.VBox(children=widget_list)
display(vbox)

In [ ]:
z = np.zeros((nz, nz))
z[np.arange(len(z)), np.arange(len(z))] = 1
z = z.astype(np.float32)
z = torch.from_numpy(z)
z = z.view(z.size(0), z.size(1), 1, 1)
z = Variable(z)
x = G(z)

In [ ]:
im = grid_of_images_default(x.data.numpy(), normalize=True)
imsave('{}/latent.png'.format(dataset), im)
Image('{}/latent.png'.format(dataset))

In [ ]:
print(nz)
for i in range(1):
    z = np.random.uniform(size=(50, nz)) <= 0.5
    z = z.astype('float32')    
    z = torch.from_numpy(z)
    
    F = np.random.randint(0, nz, size=1)
    
    for f in F:
        z[:, f] = 1.
    z_ = z.view(z.size(0), z.size(1), 1, 1)
    z_ = Variable(z_)
    x1 = G(z_)
    
    for f in F:
        z[:, f] = 0.
    z_ = z.view(z.size(0), z.size(1), 1, 1)
    z_ = Variable(z_)
    x0 = G(z_)
    
    x1 = x1.data.numpy()
    x0 = x0.data.numpy()
    x = np.empty((x0.shape[0], 2) + x0.shape[1:])
    x[:, 0] = x0
    x[:, 1] = x1
    x = x.reshape((x0.shape[0] * 2,) + x.shape[2:])
    im = grid_of_images_default(x, normalize=True)
    im = (im * 255.)
    showarray(im)